In [111]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Reading the input CSV file with custom schema

In [112]:
schema = StructType([StructField("index", IntegerType(), True),StructField("product", StringType(), True),StructField("category",  StringType(), True),
StructField("sub_category",  StringType(), True),StructField("brand",  StringType(), True),StructField("sale_price",  FloatType(), True),StructField("market_price",  FloatType(), True),
StructField("type",  StringType(), True),StructField("rating",  StringType(), True),StructField("description",  StringType(), True)
])

spark = SparkSession.builder.appName("bb_discount").getOrCreate()
df_file= spark.read.schema(schema).option("header",True).option("multiLine",True).csv(r"C:\Users\mukun\Documents\datasets\big_basket_data.csv",quote='"')





# Filter null records & Dropping unwanted columns

In [113]:
df_dropped=df_file.drop("description").drop("type")
df_nonnull=df_dropped.filter(df_dropped.index.isNotNull())

# Category with highest discount average

In [114]:
df_percent=df_nonnull.withColumn("discount",func.round(((func.col("market_price")-func.col("sale_price"))/func.col("market_price"))*100,2).cast("integer"))
df_percent.groupBy("category").agg(func.round(func.avg("discount"),2).alias("avg_discount"))\
.orderBy(func.desc("avg_discount")).show(10)

+--------------------+------------+
|            category|avg_discount|
+--------------------+------------+
|Kitchen, Garden &...|       21.84|
| Fruits & Vegetables|        21.2|
|    Beauty & Hygiene|        12.3|
|Foodgrains, Oil &...|       11.71|
|Cleaning & Household|       10.58|
|           Beverages|         9.5|
|   Eggs, Meat & Fish|        7.99|
|Gourmet & World Food|        7.82|
|Bakery, Cakes & D...|        7.52|
|Snacks & Branded ...|        6.57|
+--------------------+------------+
only showing top 10 rows



# Most reputed brand

In [115]:
df_nonnull.groupBy("brand").agg(func.round(func.avg("rating"),2).alias("avg_rating"),func.count("brand").alias("frequency"))\
.orderBy(func.desc("frequency"),func.desc("avg_rating")).show(10)

+----------------+----------+---------+
|           brand|avg_rating|frequency|
+----------------+----------+---------+
|          Fresho|      4.03|      638|
|        bb Royal|      4.05|      539|
|         BB Home|      4.11|      428|
|              DP|       4.2|      250|
|Fresho Signature|      4.21|      171|
|        bb Combo|      4.03|      168|
|            Amul|      4.04|      153|
|         INATUR |      3.81|      146|
|        Himalaya|      4.14|      141|
|           Dabur|      4.11|      138|
+----------------+----------+---------+
only showing top 10 rows



# Highest revenue category

In [116]:
df_nonnull.groupBy("category").agg(func.sum("sale_price").alias("revenue"),func.count("category").alias("frequency"))\
.orderBy(func.desc("revenue")).show()

+--------------------+------------------+---------+
|            category|           revenue|frequency|
+--------------------+------------------+---------+
|    Beauty & Hygiene| 3293749.239210129|     7867|
|Kitchen, Garden &...|1816938.1201782227|     3580|
|Gourmet & World Food|1500115.3102817535|     4690|
|Cleaning & Household| 605013.0897493362|     2675|
|Foodgrains, Oil &...| 516916.2300491333|     2676|
|Snacks & Branded ...|364675.08014678955|     2814|
|           Baby Care|326317.17083358765|      610|
|           Beverages|212186.66980171204|      885|
|Bakery, Cakes & D...| 121525.1399898529|      851|
|   Eggs, Meat & Fish|101114.12001800537|      350|
| Fruits & Vegetables| 28345.35996747017|      557|
+--------------------+------------------+---------+

